# MobileNetV2 Training - Mediapipe Style

This notebook demonstrates MobileNetV2 training and real-time inference for sign language recognition, styled and optimized like the Mediapipe notebooks. It includes:
- GPU detection/configuration
- Mixed precision logic (enabled if supported)
- Batch size and memory tips
- Training/evaluation summaries
- Progress bars for long steps
- Real-time camera input and visualization

In [ ]:
# Section 1: Install and Import Required Libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

# For GPU detection/configuration
from tensorflow.keras import mixed_precision
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import (GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Input)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE

print('✓ All libraries imported successfully')

In [ ]:
# Section 2: GPU Detection and Configuration
print('='*60)
print('🔍 GPU DETECTION & CONFIGURATION')
print('='*60)
gpus = tf.config.list_physical_devices('GPU')
print(f'Found GPUs: {gpus}')
USE_GPU = False
DEVICE = '/CPU:0'
if gpus:
    try:
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        USE_GPU = True
        DEVICE = '/GPU:0'
        print(f'✅ GPU configured: {gpus[0]}')
    except RuntimeError as e:
        print('⚠️  GPU config error:', e)
# Mixed precision (optional and beneficial on modern GPUs)
try:
    if USE_GPU:
        policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_global_policy(policy)
        print('⚡ Mixed precision enabled:', policy.name)
except Exception as e:
    print('⚠️  Mixed precision not enabled:', e)
print('Configuration complete. Using device:', DEVICE)

## Batch Size and Memory Tips
- If you get 'Out of Memory' errors, reduce `BATCH_SIZE` or disable mixed precision.
- Monitor GPU usage with `nvidia-smi -l 1` in a separate terminal.
- Close other GPU-intensive applications during training for best performance.

In [ ]:
# Section 3: Load and Preprocess Mobile Camera Input
# For demo, use sample video or webcam
VIDEO_SOURCE = 0  # 0 for webcam, or path to video file
IMG_SIZE = 128

cap = cv2.VideoCapture(VIDEO_SOURCE)
frames = []
print('Capturing 10 frames for preprocessing demo...')
for i in tqdm(range(10), desc='Capturing frames'):
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)
cap.release()
frames = np.array(frames)
print(f'Captured frames shape: {frames.shape}')

In [ ]:
# Section 4: Initialize and Configure MobileNetV2 Model
with tf.device(DEVICE):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model.trainable = False
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(10, activation='softmax', dtype='float32')(x)  # 10 classes for demo
    model = Model(inputs, outputs)
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    print('Model summary:')
    model.summary()
print(f'Model will train on device: {DEVICE}')

In [ ]:
# Section 5: Run Real-Time Inference on Mobile Frames
# For demo, run inference on captured frames
print('Running inference on demo frames...')
preds = []
for frame in tqdm(frames, desc='Inference'):
    input_tensor = np.expand_dims(frame, axis=0).astype('float32')
    with tf.device(DEVICE):
        pred = model.predict(input_tensor, verbose=0)
    preds.append(pred)
preds = np.array(preds)
print(f'Predictions shape: {preds.shape}')

In [ ]:
# Section 6: Visualize Detection Results
# For demo, overlay random predictions on frames
plt.figure(figsize=(12, 8))
for i in range(len(frames)):
    ax = plt.subplot(2, 5, i+1)
    img = frames[i]
    plt.imshow(img)
    pred_class = np.argmax(preds[i][0])
    plt.title(f'Pred: {pred_class}')
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Section 7: Save or Export Processed Frames
# For demo, save annotated frames to disk
os.makedirs('output_frames', exist_ok=True)
for i, img in enumerate(frames):
    out_path = f'output_frames/frame_{i}_pred_{np.argmax(preds[i][0])}.png'
    cv2.imwrite(out_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
print('Saved annotated frames to output_frames/')